In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField,StringType
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DateType
from pyspark.sql.functions import udf, col, array_contains, when, concat_ws, to_date
from pyspark.sql.functions import to_timestamp, lit, date_format, trim, coalesce, length
from pyspark.sql.window import Window
import pyspark.sql.functions as F

In [2]:
spark = SparkSession \
        .builder \
        .appName("Spark Covid Earlier Transformations") \
        .getOrCreate()

In [3]:
# Load dataset from Johns Hopkins University Github page
# with command pd.read_csv("datafile.csv")
#covid_daily_report = pd.read_csv("data/01-23-2020.csv")
covid_daily_report = spark.read \
                    .option("header",True) \
                    .option("inferSchema",True) \
                    .csv("data/*.csv")


#                    

In [4]:
#covid_daily_report.createOrReplaceTempView("cases_table")

In [5]:
covid_daily_report.printSchema()

root
 |-- FIPS: string (nullable = true)
 |-- Admin2: string (nullable = true)
 |-- Province_State: string (nullable = true)
 |-- Country_Region: string (nullable = true)
 |-- Last_Update: string (nullable = true)
 |-- Lat: string (nullable = true)
 |-- Long_: string (nullable = true)
 |-- Confirmed: string (nullable = true)
 |-- Deaths: string (nullable = true)
 |-- Recovered: string (nullable = true)
 |-- Active: string (nullable = true)
 |-- Combined_Key: string (nullable = true)
 |-- Incident_Rate: string (nullable = true)
 |-- Case_Fatality_Ratio: string (nullable = true)



### JHU changed the dataset schema during the time
*Initial schema: Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered*

*Second schema: FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key*

*Third schema: Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude*

*Forth schema: FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio*

In [6]:
# spark.sql("SELECT COUNT(DISTINCT 'Country/Region') AS Country, COUNT(DISTINCT 'Province/State') AS State \
#             FROM cases_table").show()

In [7]:
# print((str(covid_daily_report.columns)))
# type(covid_daily_report)
# covid_daily_report.head(5)


In [8]:
df2=covid_daily_report
df2= df2 \
        .withColumn("Country_Region", 
        when(df2["Country_Region"].contains("China"),"China")
        .otherwise(df2["Country_Region"]))

In [9]:
df2= df2 \
        .withColumn("Country_Region", 
        when(df2["Country_Region"].contains("Republic of Korea"),"Korea, South")
        .otherwise(df2["Country_Region"]))

In [10]:
df2= df2 \
        .withColumn("Country_Region", 
        when(df2["Country_Region"].contains("Cote d'Ivoir"),"Cote d Ivoir")
        .otherwise(df2["Country_Region"]))

In [11]:
df2= df2 \
        .withColumn("Country_Region", 
        when(df2["Country_Region"].contains("Taiwan*"),"Taiwan")
        .otherwise(df2["Country_Region"]))

In [12]:
df2= df2 \
        .withColumn("Combined_Key", 
        when(df2["Country_Region"].isNull(),
        concat_ws(", ", df2["Province_State"], df2["Country_Region"]))
        .otherwise(df2["Combined_Key"]))
df2= df2 \
        .withColumn("Combined_Key", 
        when(df2["Combined_Key"].contains("Taiwan*"),"Taiwan")
        .otherwise(df2["Combined_Key"]))

In [13]:
df2= df2 \
        .withColumn("CaseFatalityRatio", (df2.Deaths/df2.Confirmed)*100)

In [14]:
if not (StructField("Incidence_Rate",StringType(),True) in df2.schema):
    df2= df2 \
        .withColumn("Incidence_Rate", 
        lit(0.0))

In [15]:
# get_datetime = udf(date_convert, TimestampType())
# df2 = df2.withColumn("Date", get_datetime('Last_Update'))
def dynamic_date(col, frmts=("MM/dd/yy HH:mm", "yyyy-MM-dd HH:mm:ss")):
    return coalesce(*[to_timestamp(col, i) for i in frmts])

In [16]:
# df2 = df2 \
#         .withColumn("Last_up", to_timestamp("Last_Update", "yyyy-MM-dd HH:mm:ss"))
#         #.drop("Last_up")

In [17]:
# df2 = df2 \
#         .withColumn("Last_up",dynamic_date(col("Last_Update")))

df2 = df2.withColumn('Last_date', F.col('Last_Update').cast("timestamp")) 

In [18]:
df2 = df2.withColumn('Last_date', F.split("Last_Update", " ").getItem(0))

In [19]:
# df2 = df2 \
#         .withColumn("Last_date", 
#         when(length(df2.Last_date)<10,to_date("Last_date", "MM/d/yy"))
#         .otherwise(to_date("Last_date", "yyyy-MM-dd")))

df2=df2.filter(length(df2.Last_date)==10)


In [20]:
df2.tail(10)

[Row(FIPS=None, Admin2=None, Province_State=None, Country_Region='United Arab Emirates', Last_Update='2020-03-31 23:43:43', Lat='23.424076', Long_='53.847818000000004', Confirmed='664', Deaths='6', Recovered='61', Active='597', Combined_Key='United Arab Emirates', Incident_Rate=None, Case_Fatality_Ratio=None, CaseFatalityRatio=0.9036144578313252, Incidence_Rate=0.0, Last_date='2020-03-31'),
 Row(FIPS=None, Admin2=None, Province_State=None, Country_Region='United Kingdom', Last_Update='2020-03-31 23:43:43', Lat='55.3781', Long_='-3.4360000000000004', Confirmed='25150', Deaths='1789', Recovered='135', Active='23226', Combined_Key='United Kingdom', Incident_Rate=None, Case_Fatality_Ratio=None, CaseFatalityRatio=7.113320079522863, Incidence_Rate=0.0, Last_date='2020-03-31'),
 Row(FIPS=None, Admin2=None, Province_State=None, Country_Region='Uruguay', Last_Update='2020-03-31 23:43:43', Lat='-32.5228', Long_='-55.7658', Confirmed='338', Deaths='1', Recovered='41', Active='296', Combined_Key='

In [22]:
df2 = df2 \
        .withColumn("Last_up", to_date("Last_date", "yyyy-MM-dd"))
        #.drop("Last_up")

In [23]:
# df2 = df2 \
#         .withColumn("Last_up", 
#         when(df2["Last_up"].isNull(), to_timestamp("Last_Update", "MM/d/yy HH:mm"))
#         .otherwise(df2["Last_up"]))

In [24]:
df2 = df2 \
        .withColumn("Year", date_format(col("Last_up"), "yyyy")) \
        .withColumn("Month", date_format(col("Last_up"), "MM")) \
        .withColumn("Day", date_format(col("Last_up"), "dd")) \
        .withColumn("Datekey", date_format(col("Last_Up"), "yyyyMMdd")) \
        .withColumn("newDate", date_format(col("Last_Up"), "yyyy-MM-dd"))
        #.drop("Last_up")

In [25]:
df2 = df2 \
        .withColumn("Combined_Key2", 
                    trim(concat_ws(', ',trim(df2['Province_State']),trim(df2['Country_Region']))))

In [26]:
df2.printSchema()

root
 |-- FIPS: string (nullable = true)
 |-- Admin2: string (nullable = true)
 |-- Province_State: string (nullable = true)
 |-- Country_Region: string (nullable = true)
 |-- Last_Update: string (nullable = true)
 |-- Lat: string (nullable = true)
 |-- Long_: string (nullable = true)
 |-- Confirmed: string (nullable = true)
 |-- Deaths: string (nullable = true)
 |-- Recovered: string (nullable = true)
 |-- Active: string (nullable = true)
 |-- Combined_Key: string (nullable = true)
 |-- Incident_Rate: string (nullable = true)
 |-- Case_Fatality_Ratio: string (nullable = true)
 |-- CaseFatalityRatio: double (nullable = true)
 |-- Incidence_Rate: double (nullable = false)
 |-- Last_date: date (nullable = true)
 |-- Last_up: date (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Day: string (nullable = true)
 |-- Datekey: string (nullable = true)
 |-- newDate: string (nullable = true)
 |-- Combined_Key2: string (nullable = false)



In [ ]:
test = df2.filter(F.col("Country_Region").isin(['Afghanistan']))
#print(df2.loc[df2['Country_Region'] == 'Afghanistan'])

In [ ]:
test.tail(5)

In [27]:
df2.head(1)

[Row(FIPS=None, Admin2=None, Province_State=None, Country_Region='Afghanistan', Last_Update='2021-02-27 05:22:28', Lat='33.93911', Long_='67.709953', Confirmed='55696', Deaths='2442', Recovered='49285', Active='3969', Combined_Key='Afghanistan', Incident_Rate='143.0731404659654', Case_Fatality_Ratio='4.384515943694341', CaseFatalityRatio=4.384515943694341, Incidence_Rate=0.0, Last_date=datetime.date(2021, 2, 27), Last_up=datetime.date(2021, 2, 27), Year='2021', Month='02', Day='27', Datekey='20210227', newDate='2021-02-27', Combined_Key2='Afghanistan')]

In [28]:
df_final = df2.select(
                 col('Datekey'), \
                 col('Year'), \
                 col('Month'), \
                 col('Day'), \
                 col('Province_State').alias('State'), \
                 col('Country_Region').alias('Country'), \
                 col('newDate'), \
                 col('Lat').alias('Latitude'), \
                 col('Long_').alias('Longitude'), \
                 col('Confirmed'), \
                 col('Deaths'), \
                 col('Recovered'), \
                 col('Active'), \
                 col('Incidence_Rate'), \
                 col('CaseFatalityRatio'), \
                 col('Combined_Key2'))

In [29]:
df_final = df_final.orderBy(['Country','State','Datekey'], ascending=True)

In [30]:
df_final.printSchema()

root
 |-- Datekey: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Day: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- newDate: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Confirmed: string (nullable = true)
 |-- Deaths: string (nullable = true)
 |-- Recovered: string (nullable = true)
 |-- Active: string (nullable = true)
 |-- Incidence_Rate: double (nullable = false)
 |-- CaseFatalityRatio: double (nullable = true)
 |-- Combined_Key2: string (nullable = false)



In [31]:
df_final.head(10)

[Row(Datekey='20200324', Year='2020', Month='03', Day='24', State=None, Country='Afghanistan', newDate='2020-03-24', Latitude='33.93911', Longitude='67.709953', Confirmed='43', Deaths='1', Recovered='1', Active='72', Incidence_Rate=0.0, CaseFatalityRatio=2.3255813953488373, Combined_Key2='Afghanistan'),
 Row(Datekey='20200325', Year='2020', Month='03', Day='25', State=None, Country='Afghanistan', newDate='2020-03-25', Latitude='33.93911', Longitude='67.709953', Confirmed='76', Deaths='2', Recovered='2', Active='80', Incidence_Rate=0.0, CaseFatalityRatio=2.631578947368421, Combined_Key2='Afghanistan'),
 Row(Datekey='20200326', Year='2020', Month='03', Day='26', State=None, Country='Afghanistan', newDate='2020-03-26', Latitude='33.93911', Longitude='67.709953', Confirmed='80', Deaths='3', Recovered='2', Active='88', Incidence_Rate=0.0, CaseFatalityRatio=3.75, Combined_Key2='Afghanistan'),
 Row(Datekey='20200327', Year='2020', Month='03', Day='27', State=None, Country='Afghanistan', newDa

In [ ]:
# output_schema = StructType() \
#       .add("State",StringType(),True) \
#       .add("Country",StringType(),True) \
#       .add("Date",DateType(),True) \
#       .add("Latitude",DoubleType(),True) \
#       .add("Longitude",DoubleType(),True) \
#       .add("Confirmed",IntegerType(),True) \
#       .add("Deaths",IntegerType(),True) \
#       .add("Recovered",IntegerType(),True) \
#       .add("Active",IntegerType(),True) \
#       .add("IncidentRate",DoubleType(),True) \
#       .add("CaseFatalityRatio",DoubleType(),True) \
#       .add("CombinedKey",StringType(),True)

In [ ]:
# df2.write \
#     .mode("overwrite") \
#     .partitionBy("Year", "Month", "Day", "Country") \
#     .parquet('out/curated/')

In [33]:
df_final.write \
    .format("com.databricks.spark.csv") \
    .mode("overwrite") \
    .option("header",False) \
    .option("escape", "") \
    .option("quote", "") \
    .option("emptyValue", "") \
    .option("delimiter", ";") \
    .save("out/processed")